# Segformer

Research Paper: https://arxiv.org/abs/2105.15203

Datasets: https://data.mendeley.com/datasets/8gf9vpkhgy/2

Implementation adapted from:
1. https://github.com/NVlabs/SegFormer
2. https://debuggercafe.com/road-segmentation-using-segformer/
3. https://www.kaggle.com/code/andrewkettle/pytorch-segformer-and-sam-on-kindey-1
4. https://medium.com/geekculture/semantic-segmentation-with-segformer-2501543d2be4

In [30]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor, SegformerConfig
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, recall_score, jaccard_score, precision_score

## Section 1: Datasets Processing

In [31]:
class Process_Datasets(Dataset):
    def __init__(self, root_dir, feature_extractor, transforms=None):
        self.root_dir = root_dir
        self.feature_extractor = feature_extractor
        self.transforms = transforms

        self.image_path = os.path.join(self.root_dir, "img")
        self.mask_path = os.path.join(self.root_dir, "mask")

        img_files = []
        for root, dirs, files in os.walk(self.image_path):
            img_files.extend(files)
        self.images = sorted(img_files)

        mask_files = []
        for root, dirs, files in os.walk(self.mask_path):
            mask_files.extend(files)
        self.masks = sorted(mask_files)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image_path = os.path.join(self.image_path, self.images[index])
        mask_path = os.path.join(self.mask_path, self.masks[index])

        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if self.transforms:
            transform = self.transforms(image=image, mask=mask)
            encoded = self.feature_extractor(transform['image'], transform['mask'], return_tensors="pt")
        else:
            encoded = self.feature_extractor(image, mask, return_tensors="pt")

        for k,v in encoded.items():
            encoded[k].squeeze_()

        return encoded

In [32]:
def load_datasets(root_dir):
    batch_size=4
    feature_extractor = SegformerFeatureExtractor(align=False, reduce_zero_label=False)    
    transform = A.Compose([
        A.Resize(128, 128),
        ToTensorV2()
    ], is_check_shapes=False)

    dataset = Process_Datasets(root_dir=root_dir, feature_extractor=feature_extractor, transforms=transform)
    train, val = train_test_split(dataset, test_size=0.2, random_state=5)

    train_dataset = DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataset = DataLoader(val, batch_size=batch_size, shuffle=True)

    return train_dataset, val_dataset

### Part 1: Darwin Dataset

In [33]:
darwin_train, darwin_val = load_datasets(root_dir="./Datasets/Darwin")
len(darwin_train), len(darwin_val)

(1221, 306)

### Part 2: Shenzhen Dataset

In [34]:
shenzhen_train, shenzhen_val = load_datasets(root_dir="./Datasets/Shenzhen")
len(shenzhen_train), len(shenzhen_val)

(113, 29)

### Part 3: Covid-19 Dataset

In [35]:
covid_train, covid_val = load_datasets(root_dir="./Datasets/COVID-19/COVID")
len(covid_train), len(covid_val)

(723, 181)

## Section 2: Model Implementation

In [38]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def train_model(train_data, val_data):
    epochs = 6
    learning_rate = 0.0025

    config = SegformerConfig(num_labels=1)
    model = SegformerForSemanticSegmentation.from_pretrained('nvidia/mit-b0', config=config)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    val_metrics = []

    # Train network
    for ep in range(epochs):
        train_loss = []
        val_loss = []

        model.train()
        for index, batch in enumerate(tqdm(train_data)):
            image = batch["pixel_values"]
            mask = batch["labels"]
            optimizer.zero_grad()
            
            outputs = model(pixel_values=image, labels=mask)
            loss = outputs.loss

            train_loss.append(loss.item())
            optimizer.step()

        model.eval()
        with torch.no_grad():
            for index, batch in enumerate(tqdm(val_data)):
                image = batch["pixel_values"]
                mask = batch["labels"]
                optimizer.zero_grad()

                outputs = model(pixel_values=image, labels=mask)
                logits = F.interpolate(outputs.logits, size=mask.shape[-2:], mode="bilinear", align_corners=False)
                prediction = logits.argmax(dim=1)

                for pred, true in zip(prediction, mask):
                    pred_mask = pred.cpu().numpy()
                    true_mask = true.cpu().numpy()

                    iou = jaccard_score(true_mask.flatten(), pred_mask.flatten(), average='weighted')
                    accuracy = accuracy_score(true_mask.flatten(), pred_mask.flatten())
                    precision = precision_score(true_mask.flatten(), pred_mask.flatten(), average='weighted')
                    recall = recall_score(true_mask.flatten(), pred_mask.flatten(), average='weighted')
                    f1 = f1_score(true_mask.flatten(), pred_mask.flatten(), average='weighted')

                    val_metrics.append([iou, accuracy, precision, recall, f1])

                loss = outputs.loss
                val_loss.append(loss.item())
                optimizer.step()

        print(f"Epoch [{ep+1}/{epochs}]. Training Loss [{np.mean(train_loss)}]. Validation Loss [{np.mean(val_loss)}]")

    metrics = pd.DataFrame(val_metrics, columns=["IoU", "Accuracy", "Precision", "Recall", "F1"])
    return model, metrics

### Part 1: Darwin Dataset

In [39]:
darwin_model, darwin_metrics = train_model(darwin_train, darwin_val)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 306/306 [04:16<00:00,  1.19it/s]


Epoch [1/6]. Training Loss [0.4805369806475175]. Validation Loss [0.47518783427920996]


100%|██████████| 306/306 [04:17<00:00,  1.19it/s]


Epoch [2/6]. Training Loss [0.4808707416643382]. Validation Loss [0.4776922834464927]


100%|██████████| 306/306 [04:15<00:00,  1.20it/s]


...

Epoch [4/6]. Training Loss [0.4798344814757848]. Validation Loss [0.4787420781414493]


100%|██████████| 306/306 [04:15<00:00,  1.20it/s]


Epoch [5/6]. Training Loss [0.480380810315154]. Validation Loss [0.4792502614018185]


100%|██████████| 306/306 [04:15<00:00,  1.20it/s]

Epoch [6/6]. Training Loss [0.48025338911893034]. Validation Loss [0.4773622329523361]


### Part 2: Shenzhen Dataset

In [40]:
shenzhen_model, shenzhen_metrics = train_model(shenzhen_train, shenzhen_val)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 29/29 [00:24<00:00,  1.20it/s]


Epoch [1/6]. Training Loss [0.5650780723158237]. Validation Loss [0.5577489343182794]


100%|██████████| 29/29 [00:24<00:00,  1.20it/s]


Epoch [2/6]. Training Loss [0.5646604542183665]. Validation Loss [0.5639664736287348]


100%|██████████| 29/29 [00:24<00:00,  1.20it/s]


...

Epoch [4/6]. Training Loss [0.5638870449192757]. Validation Loss [0.5590729507906683]


100%|██████████| 29/29 [00:23<00:00,  1.21it/s]


Epoch [5/6]. Training Loss [0.5640107298319319]. Validation Loss [0.5601615679675135]


100%|██████████| 29/29 [00:23<00:00,  1.21it/s]

Epoch [6/6]. Training Loss [0.5639024250275266]. Validation Loss [0.5611148641027254]


### Part 3: Covid-19 Dataset

In [41]:
covid_model, covid_metrics = train_model(covid_train, covid_val)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 181/181 [03:04<00:00,  1.02s/it]


Epoch [1/6]. Training Loss [0.5070195704245138]. Validation Loss [0.5091175147183034]


100%|██████████| 181/181 [02:58<00:00,  1.01it/s]


Epoch [2/6]. Training Loss [0.507060460365]. Validation Loss [0.5010465138195628]


100%|██████████| 181/181 [02:59<00:00,  1.01it/s]


...

Epoch [4/6]. Training Loss [0.5069860616907539]. Validation Loss [0.5097301406425666]


100%|██████████| 181/181 [03:02<00:00,  1.01s/it]


Epoch [5/6]. Training Loss [0.5070240113827532]. Validation Loss [0.5061063389422485]


100%|██████████| 181/181 [03:04<00:00,  1.02s/it]

Epoch [6/6]. Training Loss [0.5068797120002625]. Validation Loss [0.5008786123101883]


In [42]:
darwin_metrics.to_csv("./results/darwin_segformer.csv", index=False)
shenzhen_metrics.to_csv("./results/shenzhen_segformer.csv", index=False)
covid_metrics.to_csv("./results/covid_segformer.csv", index=False)

## Section 3: Model Evaluation

In [43]:
def evaluation(metric):
    avg_iou = metric['IoU'].mean()
    avg_accur = metric['Accuracy'].mean()
    avg_prec = metric['Precision'].mean()
    avg_recall = metric['Recall'].mean()
    avg_f1 = metric['F1'].mean()

    print(f"Validation Metrics: IoU: {avg_iou}, Accuracy: {avg_accur}, Precision: {avg_prec}, Recall: {avg_recall}, F1 Score: {avg_f1}")

### Part 1: Darwin Dataset

In [45]:
evaluation(darwin_metrics)

Validation Metrics: IoU: 0.4574702863784205, Accuracy: 0.6720253902254245, Precision: 0.4574702863784205, Recall: 0.6720253902254245, F1 Score: 0.542628702175488


### Part 2: Shenzhen Dataset

In [46]:
evaluation(shenzhen_metrics)

Validation Metrics: IoU: 0.561202498250886, Accuracy: 0.7467041015625, Precision: 0.561202498250886, Recall: 0.7467041015625, F1 Score: 0.6397997846948161


### Part 3: Covid-19 Dataset

In [47]:
evaluation(covid_metrics)

Validation Metrics: IoU: 0.5670235187731149, Accuracy: 0.7491341269477296, Precision: 0.5670235187731149, Recall: 0.7491341269477296, F1 Score: 0.6439514189280622
